## Подключение библиотек

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

## Получение информаций

In [4]:
url = 'https://habr.com/ru/all/' # страница со всеми статьями 
page = requests.get(url)

In [7]:
page.status_code

200

In [9]:
soup = bs(page.text, 'html.parser')

In [11]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n\n  <head>\n    <title>Все статьи подряд &#x2F; Хабр</title>\n<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">\n<link href="https://habr.com/ru/articles/" rel="canonical" data-hid="e3fa780">\n<link href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate" data-hid="7d51b8a">\n<link href="https://habr.com/en/articles/" hreflang="en" rel="alternate" data-hid="7d51b8a">\n<meta itemprop="image" content="/img/habr_ru.png">\n<meta property="og:image" content="/img/habr_ru.png">\n<meta property="og:image:width" content="1200">\n<meta property="og:image:height" content="630">\n<meta property="aiturec:image" content="/img/habr_ru.png">\n<meta name="twitter:image" content="/img/habr_ru.png">\n<meta property="vk:image" content="/img/habr_ru.png?format=vk">\n<meta property="fb:app_id" content="444736788986613">\n<meta property="fb:pages" content="472597926099084">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="tw

In [13]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Все статьи подряд / Хабр</title>
<link data-hid="2a79c45" href="/img/habr_ru.png" rel="image_src"/>
<link data-hid="e3fa780" href="https://habr.com/ru/articles/" rel="canonical"/>
<link data-hid="7d51b8a" href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate"/>
<link data-hid="7d51b8a" href="https://habr.com/en/articles/" hreflang="en" rel="alternate"/>
<meta content="/img/habr_ru.png" itemprop="image"/>
<meta content="/img/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="/img/habr_ru.png" property="aiturec:image"/>
<meta content="/img/habr_ru.png" name="twitter:image"/>
<meta content="/img/habr_ru.png?format=vk" property="vk:image"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@habr_com" name=

In [15]:
# словарь даных
result_list = {'title': [], 'namecompany': [], 'description': [], 'rating': [], 'field': [], 'date': [], 'textpub': []}

## Получаем данные

In [18]:
pagenum = 1
for i in range(10):
    url = 'https://habr.com/ru/articles/page' + str(pagenum) + '/' # переход на ссылуку с определённым номером сраницы
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h2', class_='tm-title tm-title_h2')# получаем заголовки всех статей на этой странице
    
    for i in titles: 
        # переход на страницу статьи
        url = 'https://habr.com' + str(i.a.get('href')) 
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        
        name_company = soup.find('a', class_='tm-company-snippet__title')# получаем название компаний
        desc_company = soup.find('div', class_='tm-company-snippet__description')# получаем описание компаний
        
        if (name_company is not None): #если на странице присутсвует компания
        
            result_list['title'].append(i.text) # записываем название статьи
            result_list['namecompany'].append(name_company.text) # записываем название компании
            result_list['description'].append(desc_company.text) # записываем описание компании
            
            datepub = soup.find('span', class_='tm-article-datetime-published') # находим дату публикаций
            result_list['date'].append(datepub.time['datetime'][0: 10]) # записываем дату публикаций
            
            # текст статьи
            try:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            except:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            result_list['textpub'].append(textpub)
            
            # переход на страницу компании
            url = 'https://habr.com' + str(name_company.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            
            #записываем рейтинг
            rating = soup.find('span', class_='tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating')
            if(rating is None):
                result_list['rating'].append('0')
            else:
                result_list['rating'].append((rating.text).strip())
               
             #записываем отрасли компаний
            fieldtext = ""
            fields = soup.find_all('a', 'tm-company-profile__categories-text')
            for field in fields:
                fieldtext = fieldtext + ((field.text).strip()) + ", "
            if (fields is None):
                result_list['field'].append(None)
            else:
                result_list['field'].append(fieldtext[0:-2])
            
    pagenum += 1

In [21]:
result_list

{'title': ['Издержки микросервисов, которые ваш стартап может не потянуть',
  'Тяжеловесы, которые не взлетели: 5 провалившихся высокобюджетных видеоигр последних лет',
  'Что вы знаете о Яве?',
  'Перешагивая через века. Rise of nations: Thrones & Patriots',
  'Деньги и автономия — ситуация в сфере баз данных уязвимостей',
  'Сказ о том, как один программист себе мебель из ДСП на PHP программировал',
  'pytest.raises: ловим исключения правильно',
  '«Сети Судного дня». Как разработка противоядерной ПВО привела к рождению Интернета?',
  'RISC-V против ARM: обзор свежих одноплатников мая 2025 года',
  'В Политехнический музей будет передана одна из последних сохранившихся ЕС ЭВМ',
  '5 смартфонов, ломающих привычные представления о мобильной технике',
  'Как писать промпты для генерации изображений: часть 2',
  'Как писать промпты для генерации изображений: часть 1',
  'GUI-приложения с помощью Python-Tkinter',
  'Визуальное проектирование управляющей логики фитнес-браслета',
  '«Они пр

In [23]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
title - 0
namecompany - 0
description - 0
rating - 0
field - 0
date - 0
textpub - 0


## Сохранение даных

In [25]:
df = pd.DataFrame(data=result_list)


In [27]:
df.head()

,title,namecompany,description,rating,field,date,textpub
0,"Издержки микросервисов, которые ваш стартап мо...",RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,0,"Связь и телекоммуникации, Домены и хостинг, Ве...",2025-05-18,"Выживание стартапа зависит от того, наскольк..."
1,"Тяжеловесы, которые не взлетели: 5 проваливших...",МТС,Про жизнь и развитие в IT,0,"Связь и телекоммуникации, Мобильные технологии...",2025-05-18,"Привет, Хабр! Меня зовут Настя, я автор команд..."
2,Что вы знаете о Яве?,Axiom JDK,на страже безопасности Java,0,"Программное обеспечение, Консалтинг и поддержк...",2025-05-18,23 мая этого года языку Java исполняется 30 ле...
3,Перешагивая через века. Rise of nations: Thron...,Timeweb Cloud,То самое облако,0,Связь и телекоммуникации,2025-05-17,Так-так-так... что это здесь у нас? Мои глаза ...
4,Деньги и автономия — ситуация в сфере баз данн...,VAS Experts,Разработчик платформы глубокого анализа трафика,0,"Программное обеспечение, Аппаратное обеспечени...",2025-05-17,Недавно в США задумались о сокращении финансир...


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        88 non-null     object
 1   namecompany  88 non-null     object
 2   description  88 non-null     object
 3   rating       88 non-null     object
 4   field        88 non-null     object
 5   date         88 non-null     object
 6   textpub      88 non-null     object
dtypes: object(7)
memory usage: 4.9+ KB


In [31]:
df.to_csv('MyHabrCsv.csv')